In [1]:
import cv2
import time
import mediapipe as mp
import handTrackingModule as htm
import autopy
import numpy as np

In [4]:

pTime = 0
cTime = 0
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,640)
frameR = 100
detector = htm.handDetector(maxHands=1)
wScr, hScr = autopy.screen.size()
while True:
    success, imgg = cap.read()
    img = cv2.flip(imgg,1)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img)
    # 2. Get the tip of the index and middle fingers
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
        #print(x1, y1, x2, y2)
        fingers = detector.fingersUp()
        cv2.rectangle(img,(frameR,frameR),(640-frameR,640-frameR),(255,0,255),2)
        #print(fingers)
        if fingers[1]==1 and fingers[2]==0:
            x3 = np.interp(x1,(0,640-frameR),(0,wScr))
            y3 = np.interp(y1,(0,640-frameR),(0,hScr))
            autopy.mouse.move(wScr-x3,y3)
            cv2.circle(img,(x1,y1),15,(255,0,0),cv2.FILLED)
        if fingers[1]==1 and fingers[2]==1:
            length, img,lineinfo = detector.findDistance(8,12,img)
            if length<40:
                cv2.circle(img,(lineinfo[4],lineinfo[5]),15,(0,255,0),cv2.FILLED)
                autopy.mouse.click()
            
    
    # 11. Frame Rate
    
    pTime = cTime
    cv2.putText(img, str(int(fps)), (20, 50), cv2.FONT_HERSHEY_PLAIN, 3,
    (255, 0, 0), 3)
    # 12. Display
    cv2.imshow("Image", img)
    #cv2.waitKey(1)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
# Destroy all the windows
cv2.destroyAllWindows()
    